In [ ]:
# Estructura de archivos para la práctica 3 de Análisis de Grafos (tema: red de personajes Marvel)

# Justificación: Se usa la API de Marvel para construir un grafo realista basado en cómics y personajes.
# Cada personaje será un nodo, y los enlaces entre ellos se crean si aparecen en el mismo cómic.

# 01_extractor.ipynb (descarga datos desde la API de Marvel)
import requests
import pandas as pd
import time

# Claves de acceso Marvel (guardar en marvel_keys.txt con formato: PUBLIC_KEY,PRIVATE_KEY,HASH)
with open("marvel_keys.txt", "r") as f:
    keys = f.read().strip().split(",")
    PUBLIC_KEY = keys[0]
    HASH = keys[2]

BASE_URL = "https://gateway.marvel.com/v1/public"
ts = 1

comics = []

for offset in range(0, 100, 20):
    url = f"{BASE_URL}/comics?orderBy=-modified&limit=20&offset={offset}&ts={ts}&apikey={PUBLIC_KEY}&hash={HASH}"
    response = requests.get(url)
    if response.status_code != 200:
        continue
    results = response.json()["data"]["results"]
    for comic in results:
        comic_id = comic["id"]
        title = comic["title"]
        char_url = f"{BASE_URL}/comics/{comic_id}/characters?ts={ts}&apikey={PUBLIC_KEY}&hash={HASH}"
        char_response = requests.get(char_url)
        if char_response.status_code != 200:
            continue
        characters = char_response.json()["data"]["results"]
        for char in characters:
            comics.append({
                "comic": title,
                "character": char["name"]
            })
    time.sleep(1)  # para evitar sobrecarga

# Guardar en INPUT
marvel_df = pd.DataFrame(comics)
marvel_df.to_csv("INPUT/marvel_characters.csv", index=False)